## Build a simple LLM application using LCEL

In this quickstart, we will build a simple LLM application using Langchain. This application will translate text from English to another language. This is relatively simle LLM application - it is just a single LLM call plus some prompting. 

High level overview:
- Using language models
- Using PromptTemplates and OutputParsers
- Using Langchain Expression Language (LCEL) to chain components together
- Debugging and tracing the application in LangSmith
- Deploy the application in Langserve

For this tutorial we will use [Groq LPU](https://groq.com/blog/the-groq-lpu-explained). For that reasong we need Groq API key, which can be created in [Groq API Keys](https://console.groq.com/home?utm_source=website&utm_medium=outbound_link&utm_campaign=dev_console_click). To call Groq we are using [**LP AI Inferencing Engine**](https://groq.com/blog/the-groq-lpu-explained). 

But Why GROQ?

1. Groq is a Fast AI Inference. 
2. Uses LPU. LPU stands for Language Processing Unit 
3. LPU Inference Engine is a hardware and software platform that delivers exceptional compute speed, quality and enery efficiency. 
4. LPU is designed to overcome 2 LLM bottlenecks - compute density and memory bandwidth. LPU has greater compute capacity than GPU in regards to LLMs. 

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.environ['GROQ_API_KEY']
print(f'Groq API Key: {GROQ_API_KEY[:3]}**{GROQ_API_KEY[-3:]}')

Groq API Key: gsk**qNU


Initialize **ChatGroq**. To initialize ChatGroq we need to pass the model name. All the model names can be found in [Groq docs](https://console.groq.com/docs/models).

In [10]:
from langchain_groq import ChatGroq

model = ChatGroq(model='Gemma2-9b-It', api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1229b0a50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1229b1450>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

When we chat with the LLM model, we should specify the LLM model which message is provided by the human and which message is kind of **instruction** to the LLM. 

- **SystemMessage** - instruction how the LLM should behave

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Translate provided sentence from {sourceLanguage} to {targetLanguage}'),
    ('user', '{input}')
])

chain = prompt | model
chain

ChatPromptTemplate(input_variables=['input', 'sourceLanguage', 'targetLanguage'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['sourceLanguage', 'targetLanguage'], input_types={}, partial_variables={}, template='Translate provided sentence from {sourceLanguage} to {targetLanguage}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1229b0a50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1229b1450>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
response = chain.invoke(input={
    'sourceLanguage': 'English',
    'targetLanguage': 'Bulgarian',
    'input': 'Hello, How are you?'
})
response

AIMessage(content='Здравей, как си? (Zdravey, kak si?) \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 22, 'total_tokens': 39, 'completion_time': 0.030909091, 'prompt_time': 0.001324411, 'queue_time': 0.192357574, 'total_time': 0.032233502}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--40795bd1-3be6-4eb5-aed4-b465d6e93000-0', usage_metadata={'input_tokens': 22, 'output_tokens': 17, 'total_tokens': 39})

Another way of calling it 

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content='Translate this from English to Bulgarian'),
    HumanMessage(content='Hey, How are you?')
]

response = model.invoke(messages)
response

AIMessage(content='Здравей, как си? (Zdraвей, kak si?) \n\n\nThis is the most common and informal way to say "Hey, how are you?" in Bulgarian. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 21, 'total_tokens': 60, 'completion_time': 0.070909091, 'prompt_time': 0.001323241, 'queue_time': 0.190184415, 'total_time': 0.072232332}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--33073df2-6785-433d-bd1d-9bbb7160b52b-0', usage_metadata={'input_tokens': 21, 'output_tokens': 39, 'total_tokens': 60})

In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
response_str = output_parser.invoke(response)
response_str

'Здравей, как си? (Zdraвей, kak si?) \n\n\nThis is the most common and informal way to say "Hey, how are you?" in Bulgarian. \n'

#### LCEL - Langchain Expression Language

Let's do this using Langchain Expression Language (**LCEL**). Using LCEL we can chain the components. 

In [15]:
chain = model | output_parser

chain.invoke(messages)

'Здравей, как си? (Zdravey, kak si?)\n\n\nThis is the most common and informal way to say "Hey, How are you?" in Bulgarian. \n'

Using ChatPromptTemplate

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(messages=[
    ('system', 'Translate this from {srcLang} to {tgtLang} in most informal way. Try to use some swag. Just answer, without explanation'),
    ('user', '{input}')
])

output_parser = StrOutputParser()

chain = prompt_template | model | output_parser

response = chain.invoke(input={
    'srcLang': 'English',
    'tgtLang': 'Bulgarian',
    'input': 'I need more money'
})

response

'Нужно ми паца, пари! 💸\n'